In [ ]:
from notebooks import *

In [ ]:
%%time
joypy.joyplot(
    # Perf: kind='kde' and kind='counts' are roughly the same
    # kind='kde',
    kind='counts', bins=80,
    fill=False,
    linecolor='k',
    linewidth=.5,
    xlabels=True,
    ylabels=False,
    # overlap=.001,
    # overlap=.01,
    # overlap=.1,
    # overlap=.5,
    # overlap=1,
    # XXX This gets _very slow_ with more lines
    #   - Unusable >100, and our spectro Sxx are more like ~600
    data=[
        stats.distributions.norm.rvs(mu, 10, 100)
        for mu in np.arange(50)
    ],
)
plt.show()

In [ ]:
# Here's what's required to turn an FT into a periodogram (power by freq)
#   - TODO Oops, x-axis is still off by 2x...
x = ...  # Real signal, e.g. audio waveform
sample_rate = 22050
f, Pxx = scipy.signal.periodogram(x, fs=sample_rate, scaling='spectrum')
plt.plot(f, Pxx); plt.show()
plt.plot((np.abs(np.fft.rfft(x)) / len(x))**2 * 2); plt.show()

In [ ]:
# Diff spectro vs. librosa melspectrogram
f, t, Sxx = np.abs(spectro(audio, nperseg=nperseg, overlap=overlap, detrend='constant', scaling='density', mode='complex'))**2
Sxx = Sxx * 1024**2 * 8  # Works with scaling='density'
display(Sxx.shape)
plt.hist(Sxx.flatten(), bins=100, log=True); plt.show()
Sxx = librosa.power_to_db(np.dot(mel_basis, Sxx)); display(Sxx.shape)
# plt.pcolormesh(Sxx); [s.set_visible(False) for s in plt.gca().spines.values()]; plt.show()

n_fft = nperseg
Sxx = librosa.feature.melspectrogram(x.astype(float), sample_rate, None, n_fft, int(n_fft*(1-overlap)), n_mels=n_mels, power=2)
display(Sxx.shape)
plt.hist(Sxx.flatten(), bins=100, log=True); plt.show()
Sxx = librosa.power_to_db(np.dot(mel_basis, Sxx)); display(Sxx.shape)
# plt.pcolormesh(Sxx); [s.set_visible(False) for s in plt.gca().spines.values()]; plt.show()

In [ ]:
# Diff mfcc vs. librosa mfcc
n_mfcc = 1024
plt_mfcc(x, mfcc=partial(mfcc, mels_div=2, n_mfcc=n_mfcc, std=True), show_audio=False, fancy=False)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter(''))
plt.show()

M = librosa.feature.mfcc(x.astype(float), sample_rate, n_mfcc=n_mfcc)
M = (M - M.mean(axis=1)[:, np.newaxis]) / M.std(axis=1)[:, np.newaxis]  # Standardize
plt.pcolormesh(M)
plt.gca().yaxis.set_major_formatter(mpl.ticker.FormatStrFormatter(''))
plt.gca().tick_params(labelsize=8)
[s.set_visible(False) for s in plt.gca().spines.values()]
plt.show()

In [ ]:
import dask
import dask.bag
import dask.dataframe as dd

dask.set_options(scheduler='processes')

df = pd.DataFrame({'a':range(10,20), 'b':range(20,30)}); df

ddf = dd.from_pandas(df, npartitions=8)
ddf.apply?

dask.bag.from_sequence(range(100)).compute()

In [ ]:
from datatypes import Recording
from load import *

recs_paths = load_recs_paths(['recordings-new'])
row = recs_paths.iloc[0]
rec = load_rec(row.dataset, row.path)

import dataclasses
dir(dataclasses)

# load_recs_data(recs_paths)

dataclasses.asdict(rec)
dataclasses.fields(rec)

In [ ]:
from potoo.plot import *

# repr(ggplot(mpg) + geom_bar(aes(x='class')))

theme_figsize('inline_short').rcParams

figsize('inline_short');
# figsize('inline_short', dpi=72);
# figsize('inline_short', dpi=150);

%time plt.plot(range(10)); display(plot_to_img())
%time plt.plot(range(10)); plt.show()

In [ ]:
from cache import *

@cache(version=4)
def foo(x): return x+2

foo(3)
foo(3, __cache_version=5)

In [ ]:
import pickle
import zict

f = zict.Func(pickle.dumps, pickle.loads, zict.File('/tmp/zict-0/'))
d = zict.LRU(
    n=2,
    d=f,
    on_evict=lambda k, v: print('evict: %s:%s' % (k, v)),
    weight=lambda k, v: 1,
)

In [ ]:
import pickle
import zict
from zict_util import PersistentBuffer

a = {}
b = zict.Func(pickle.dumps, pickle.loads, zict.File('/tmp/zict-0/'))
d = PersistentBuffer(
    fast=a,
    slow=b,
    n=2,
    weight=lambda k, v: 1,
    evict_fast_callbacks=[lambda k, v: print('evict_fast: %s:%s' % (k, v))],
    load_fast_callbacks=[lambda k, v: print('load_fast: %s:%s' % (k, v))],
)

show = lambda: print('fast[%s], slow[%s]' % (dict(d.fast), dict(d.slow)))

In [ ]:
show()
d['x'] = 1; show()
d['y'] = 2; show()
d['z'] = 3; show()
d['y']; show()
d['x']; show()
d['x'] = -1; show()
d['z']; show()
d['x'] = 1; show()
d['y'] = -2; show()
d['y']; show()
d['x']; show()
d['z']; show()

In [ ]:
df = DF(dict(a=[1,2,3],b=[4,5,6]))
df.loc[:2, ['a']]

In [ ]:
df = DF(dict(a=[{'b':2}]))
df.a.str['b']

In [ ]:
from pathlib import Path

list(Path('data').joinpath('xc').glob('*'))

In [ ]:
from notebooks import *

'foo.bar/baz.ext'.rsplit('.', 1)

In [ ]:
%%prun -l50 -scumtime
pp(Load().recs_paths(['xc']).shape)

In [ ]:
# TODO Debug pickle error

In [ ]:
from notebooks import *

In [ ]:
# pd.Series(
#     map_progress(lambda x: os.getpid(), range(10), use='dask', scheduler='processes', get_kwargs=dict(num_workers=7))
# ).value_counts()

In [ ]:
# load = Load()
# recs_all = load.recs(limit=500, datasets=['xc'])

In [ ]:
##
# TODO Under par: only ~15-20% total cpu on n1-standard-16
recs_paths_all = load.recs_paths(datasets=['xc'])
# load.audio_to_wav(recs_paths_all)

In [ ]:
get_figsize()

{
  'width': 12,
  'height': 8.0,
  'aspect_ratio': 0.6666666666666666,
  'dpi': 72,
  '_figure_format': 'retina',
  '_Rdefaults': '--width=12 --height=8.0 --units=in --res=144'
}

In [ ]:
import logging
logging.basicConfig(level=logging.DEBUG)
import gcsfs

gs = gcsfs.GCSFileSystem(project='bubo-1')
# gs.ls('bubo-data/v0/data/xc/data')[:10]
gs.walk('bubo-data/v0/data/xc/data/TUVU/')

In [ ]:
gcsfs.core.split_path( 'gs://bubo-data/date')

['bubo-data', 'date']

DEBUG:gcsfs.core:__del__(args=(), kwargs={})


DEBUG:gcsfs.core:close(args=(), kwargs={})


In [ ]:
with gs.open('gs://bubo-data/date', 'w') as f:
    f.write('foo')

DEBUG:gcsfs.core:open(args=('gs://bubo-data/date', 'w', None, None, None, None), kwargs={})


DEBUG:gcsfs.core:__init__(args=(<gcsfs.core.GCSFileSystem object at 0x11c102da0>, 'gs://bubo-data/date', 'wb', 5242880, None, 'none', None), kwargs={})


DEBUG:gcsfs.core:flush(args=(False,), kwargs={})


DEBUG:gcsfs.core:flush(force=False) with buffer (3) < min size (2 ** 18), skipping block upload.


DEBUG:gcsfs.core:close(args=(), kwargs={})


DEBUG:gcsfs.core:flush(args=(True,), kwargs={})


DEBUG:gcsfs.core:_simple_upload(args=(), kwargs={})


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "POST /upload/storage/v1/b/bubo-data/o?uploadType=media&name=date HTTP/1.1" 200 647


DEBUG:gcsfs.core:invalidate_cache(args=('bubo-data',), kwargs={})


In [ ]:
gs.open('gs://bubo-data/date').read()

DEBUG:gcsfs.core:open(args=('gs://bubo-data/date', 'rb', None, None, None, None), kwargs={})


DEBUG:gcsfs.core:__init__(args=(<gcsfs.core.GCSFileSystem object at 0x11c102da0>, 'gs://bubo-data/date', 'rb', 5242880, None, 'none', None), kwargs={})


DEBUG:gcsfs.core:info(args=('gs://bubo-data/date',), kwargs={})


DEBUG:gcsfs.core:_get_object(args=('gs://bubo-data/date',), kwargs={})


DEBUG:gcsfs.core:_maybe_get_cached_listing(args=('bubo-data//',), kwargs={})


...

DEBUG:gcsfs.core:_fetch_range(args=(<google.auth.transport.requests.AuthorizedSession object at 0x11c162cc0>, 0, 5242883), kwargs={})


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /download/storage/v1/b/bubo-data/o/date?generation=1530128395796535&alt=media HTTP/1.1" 206 3


DEBUG:gcsfs.core:__del__(args=(), kwargs={})


DEBUG:gcsfs.core:close(args=(), kwargs={})


b'foo'

In [ ]:
gs.glob('bubo-data/v0/data/xc/data/WIWA/**')

In [ ]:
from google.cloud import storage
client = storage.Client()
bucket = client.get_bucket('bubo-data')

In [ ]:
gcsfs.GCSFileSystem(project='bubo-1', token=client._credentials).ls('bubo-data')
dirs(client._credentials)

{
  'apply': <bound method Credentials.apply of <google.oauth2.credentials.Credentials object at 0x118f3c5f8>>,
  'before_request': <bound method Credentials.before_request of <google.oauth2.credentials.Credentials object at 0x118f3c5f8>>,
  'client_id': '764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com',
  'client_secret': 'd-FL95Q19q7MQmFpd7hHD0Ty',
  'expired': True,
  'expiry': datetime.datetime(year=2018, month=6, day=27, hour=20, minute=1, second=34, microsecond=619230),
  'from_authorized_user_file': <bound method Credentials.from_authorized_user_file of <class 'google.oauth2.credentials.Credentials'>>,
  'from_authorized_user_info': <bound method Credentials.from_authorized_user_info of <class 'google.oauth2.credentials.Credentials'>>,
  'has_scopes': <bound method ReadOnlyScoped.has_scopes of <google.oauth2.credentials.Credentials object at 0x118f3c5f8>>,
  'id_token':
    'eyJhbGciOiJSUzI1NiIsImtpZCI6IjRlZjUxMThiMDgwMGJkNjBhNDE5NDE4NmRjYjUzOGZjNjZlNWVi

DEBUG:gcsfs.core:ls(args=('bubo-data', False), kwargs={})


DEBUG:gcsfs.core:_ls(args=('bubo-data/', False), kwargs={})


DEBUG:gcsfs.core:_list_objects(args=('bubo-data/',), kwargs={})


DEBUG:gcsfs.core:_maybe_get_cached_listing(args=('bubo-data/',), kwargs={})


DEBUG:gcsfs.core:_maybe_get_cached_listing: bubo-data/


['bubo-data/date', 'bubo-data/v0/']

In [ ]:
blobs = [x for x in bucket.list_blobs(prefix='v0/data/xc/data/SOSP/',
    fields='items/name,nextPageToken',
)]
len(blobs)

DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/bubo-data/o?projection=noAcl&prefix=v0%2Fdata%2Fxc%2Fdata%2FSOSP%2F&fields=items%2Fname%2CnextPageToken HTTP/1.1" 200 60026


DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/bubo-data/o?pageToken=CiV2MC9kYXRhL3hjL2RhdGEvU09TUC8zNjI1MTQvcGFnZS5odG1s&projection=noAcl&prefix=v0%2Fdata%2Fxc%2Fdata%2FSOSP%2F&fields=items%2Fname%2CnextPageToken HTTP/1.1" 200 17664


1296

In [ ]:
gcsfs.core.split_path

gcsfs.core.split_path  # function

In [ ]:
[x.name for x in blobs]

In [ ]:
from pathlib import Path
from datasets import xc
len(list(xc.data_dir.glob('*/*/audio.mp3')))

129449

In [ ]:
from util import df_rows
from more_itertools import *
list(unique_everseen(AttrDict(row) for row in df_rows(DF(dict(a=[1,2,2], b=['foo','bar','bar'])))))

[attrdict.dictionary.AttrDict({'a': 1, 'b': 'foo'}), attrdict.dictionary.AttrDict({'a': 2, 'b': 'bar'})]

In [ ]:
xs = [1,2,3]
xs.extend(x for x in range(5))
xs

[1, 2, 3, 0, 1, 2, 3, 4]

In [ ]:
print_sys_info()

In [ ]:
from potoo.numpy import *
from util import *
np_sample_stratified(
    X=np.arange(20),
    y=list(flatten([c] * 4 for c in 'abcde')),
    # n=1,
    n=20,
    # frac=1.0,
    # frac=0.6,
)

args:
{
  'X_shape': (20,),
  'y_shape': (20,),
  'n': 20,
  'frac': 1.0,
  'replace': False
}


(
  array([ 3, 15,  9, 12, 13,  1,  5,  0, 16,  7, 10, 14, 11, 17,  8,  4,  2, 18,  6, 19]),
  array(['a', 'd', 'c', 'd', 'd', 'a', 'b', 'a', 'e', 'b', 'c', 'd', 'c', 'e', 'c', 'b', 'a', 'e', 'b', 'e'], dtype='<U1')
)

In [ ]:
from notebooks import *
ret = joblib.load(f"{artifact_dir}/20180709-211727-fd34af5-ovr_try_2/split_i=0,train=2688,test=672,classes=168,estimator=Search(n_species=168,n_recs=1.0,classifier='cls=xgbm,n_estimators=100,max_depth=100',random_state=0)/ret.pkl")
estimator = joblib.load(f"{artifact_dir}/20180709-211727-fd34af5-ovr_try_2/split_i=0,train=2688,test=672,classes=168,estimator=Search(n_species=168,n_recs=1.0,classifier='cls=xgbm,n_estimators=100,max_depth=100',random_state=0)/estimator.pkl")
sizeof = lambda x: len(joblib_dumps(x))
display(
    sizeof(estimator),
    sizeof(ret),
    [sizeof(x) for x in ret],
)

33066223

8831779

[28, 28, 6, 12, 12, 8831700, 4]

In [ ]:
from notebooks import *
estimator = joblib.load(f"{artifact_dir}/20180709-211727-fd34af5-ovr_try_2/split_i=0,train=2688,test=672,classes=168,estimator=Search(n_species=168,n_recs=1.0,classifier='cls=xgbm,n_estimators=100,max_depth=100',random_state=0)/estimator.pkl")
gbm = estimator.classifier_
gbm._Booster.dump_model('/tmp/gbm.txt', with_stats=True)  # Human readable
sizeof = lambda x: len(joblib_dumps(x))
%time stats = model_stats(gbm)
display(
    humanize.naturalsize(sizeof(gbm)),
    humanize.naturalsize(sizeof(stats)),
    df_summary(stats).T,
    stats[:20],
)

CPU times: user 1.4 s, sys: 37.6 ms, total: 1.44 s
Wall time: 1.44 s


'16.9 MB'

'1.7 MB'

,,,,,,,,min,25%,50%,75%,max
,dtype,sizeof,len,count,nunique,mean,std,,,,,
type,object,1142400,16800,16800,1,NaN,NaN,xgb,xgb,xgb,xgb,xgb
n_trees,int64,470400,16800,16800,1,NaN,NaN,16800,16800,16800,16800,16800
tree_i,int64,470396,16800,16800,16800,NaN,NaN,0,4199,8399,12599,16799
depth,int64,470392,16800,16800,30,NaN,NaN,0,5,7,9,100
node_count,int64,470400,16800,16800,42,NaN,NaN,1,17,23,29,221
leaf_count,int64,470400,16800,16800,42,NaN,NaN,1,9,12,15,111
fork_count,int64,470392,16800,16800,42,NaN,NaN,0,8,11,14,110


,type,n_trees,tree_i,depth,node_count,leaf_count,fork_count
0,xgb,16800,0,7,41,21,20
1,xgb,16800,1,8,21,11,10
2,xgb,16800,2,4,17,9,8
3,xgb,16800,3,13,33,17,16
4,xgb,16800,4,100,207,104,103
5,xgb,16800,5,9,37,19,18
6,xgb,16800,6,8,27,14,13
7,xgb,16800,7,9,27,14,13
8,xgb,16800,8,8,33,17,16
9,xgb,16800,9,5,17,9,8


In [ ]:
from notebooks import *
import pprint
(xc.metadata
    [lambda df: pd.notnull(df.species)]
    [lambda df: df.country == 'United States']
    .pipe(df_remove_unused_categories).com_name.sort_values()
    .pipe(puts, f=lambda s: len(s.unique()))
    .pipe(lambda s:
        pprint.pprint(s.unique().tolist(), indent=4, width=120, compact=True)
    )
)

In [ ]:
for s in [
    '#             f1-micro    1   .6g  $[   4 /   3]/mo  $[    /   ]/d  $[.008 / .004]/hr',
    '#             g1-small    1  1.7g  $[  13 /   5]/mo  $[    /.17]/d  $[.026 / .007]/hr',
    '#',
    '#         n1-highcpu-2    2    2g  $[  36 /  11]/mo  $[    /   ]/d  $[ .07 /  .02]/hr',
    '#         n1-highcpu-4    4    4g  $[  72 /  22]/mo  $[    /   ]/d  $[ .14 /  .03]/hr',
    '#         n1-highcpu-8    8    7g  $[ 145 /  44]/mo  $[    /   ]/d  $[ .28 /  .06]/hr',
    '#        n1-highcpu-16   16   14g  $[ 290 /  88]/mo  $[    /   ]/d  $[ .57 /  .12]/hr',
    '#        n1-highcpu-32   32   29g  $[ 580 / 175]/mo  $[    /   ]/d  $[1.13 /  .24]/hr',
    '#        n1-highcpu-64   64   58g  $[1159 / 350]/mo  $[    /   ]/d  $[2.27 /  .48]/hr',
    '#        n1-highcpu-96   96   86g  $[1739 / 526]/mo  $[ 82 / 17]/d  $[3.40 /  .72]/hr',
    '#',
    '#        n1-standard-2    2    7g  $[  49 /  15]/mo  $[  5 /  1]/d  $[ .10 /  .02]/hr',
    '#        n1-standard-4    4   15g  $[  97 /  29]/mo  $[  5 /  1]/d  $[ .19 /  .04]/hr',
    '#        n1-standard-8    8   30g  $[ 194 /  58]/mo  $[  9 /  2]/d  $[ .38 /  .08]/hr',
    '#       n1-standard-16   16   60g  $[ 388 / 116]/mo  $[ 18 /  4]/d  $[ .76 /  .16]/hr',
    '#       n1-standard-32   32  120g  $[ 776 / 233]/mo  $[ 36 /  8]/d  $[1.52 /  .32]/hr',
    '#       n1-standard-64   64  240g  $[1553 / 467]/mo  $[ 73 / 16]/d  $[3.04 /  .64]/hr',
    '#       n1-standard-96   96  360g  $[2330 / 700]/mo  $[109 / 23]/d  $[4.56 /  .96]/hr',
    '#',
    '#         n1-highmem-2    2   13g  $[  61 /  18]/mo  $[    /   ]/d  $[ .12 /  .03]/hr',
    '#         n1-highmem-4    4   26g  $[ 121 /  37]/mo  $[    /   ]/d  $[ .24 /  .05]/hr',
    '#         n1-highmem-8    8   52g  $[ 242 /  73]/mo  $[    /   ]/d  $[ .47 /  .10]/hr',
    '#        n1-highmem-16   16  104g  $[ 484 / 146]/mo  $[    /   ]/d  $[ .95 /  .20]/hr',
    '#        n1-highmem-32   32  208g  $[ 968 / 292]/mo  $[    /   ]/d  $[1.89 /  .40]/hr',
    '#        n1-highmem-64   64  416g  $[1936 / 584]/mo  $[    /   ]/d  $[3.79 /  .80]/hr',
    '#        n1-highmem-96   96  624g  $[2904 / 876]/mo  $[    /   ]/d  $[5.68 / 1.20]/hr',
]:
    if s == '#':
        print(s)
    else:
        (desc, dem_m, pre_m, dem_d, pre_d, dem_h, pre_h) = re.match(
            r'(.*?)\$\[(.*)/(.*)\]/mo.*\$\[(.*)/(.*)\]/d.*\$\[(.*)/(.*)\]/hr',
            s,
        ).groups()
        dem_h = float(dem_h.strip())
        pre_h = float(pre_h.strip())
        line = '%s$[%4.0f / %3.0f]/mo  $[%5.1f / %4.2f]/d  $[%4.3f / %4.3f]/hr' % (
            desc,
            dem_h * 24*28, pre_h * 24*28,
            dem_h * 24, pre_h * 24,
            dem_h, pre_h,
        )
        line = re.sub(r'0\.', ' .', line)
        print(line)

In [ ]:
list(map(list, np.array([1,2,4,8])[:,np.newaxis] * np.array([ 230,99, 24*.45,24*.135, .45,.135, ])))